In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
import category_encoders as ce
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


# I. Read dataset

In [ ]:
df_normalized=pd.read_csv('03_df_normalized_wonlost.csv', delimiter=',')
df_normalized.head()

,Result_offer,Amount_DE,Amount_HD,Amount_ID,Amount_OTHER,Amount_PP,Amount_PS,Amount_SP,Total_Amount,SR_codigo,Bid_Notbid,Resolution_time,Client_type,Market_segment,Client_BU,Country,Client_importance,Coverage_model,Result_offer_binary
0,Lost,0.001945,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000718,SR-2468496,Bid,0.078611,Contractor,BDZ,Power Products,ES,MA3,Sales Rep,0
1,Won,0.002045,0.0,0.1,0.0,0.02803,0.020478,0.0,0.021526,SR-2468507,Bid,0.235466,Contractor,E34,INDUSTRIAL AUTOMATION,ES,MS3,Sales Rep,1
2,Lost,0.003382,0.0,0.0,0.0,0.00000,0.000005,0.0,0.001252,SR-2468656,Bid,0.032176,Contractor,I9Z,Power Products,ES,MS3,Sales Rep,0
3,Lost,0.003382,0.0,0.0,0.0,0.00000,0.000005,0.0,0.001252,SR-2468663,Bid,0.025229,Contractor,I9Z,Power Products,ES,MS3,Sales Rep,0
4,Lost,0.003382,0.0,0.0,0.0,0.00000,0.000005,0.0,0.001252,SR-2468668,Bid,0.025229,Contractor,I9Z,Power Products,ES,MS3,Sales Rep,0


# II. Encoding - Predictors / exclude variables

Exclude Variables. Certain variables should not be used as predictors:
    * Target Variable: Exclude the Result_offer_binary column from predictors.
    * Identifiers: Columns like IDs (e.g., SR_codigo) or highly unique identifiers (e.g., invoice numbers) do not carry predictive power and should be excluded.
    * Highly Correlated Variables: If two or more variables are strongly correlated (multicollinearity), retain only one to avoid redundancy.
    * Irrelevant Variables: Columns with little relevance to the target (e.g., timestamps without transformations or descriptive text) should be excluded.

Evaluate Predictors Using Statistical Techniques
    * Correlation Analysis: Compute the correlation matrix to check how each numerical predictor correlates with the target variable. Include only variables with meaningful correlations. 

    * Feature Importance: Using a feature importance technique such as XGBoost's feature_importances_ or permutation importance to determine which features contribute most to the model.

Convert Categorical variables to numeric: Machine learning models require numeric inputs. So, categorical variables must be transformed into a numerical format to be used as predictors in the model.
    Which method?
        For Tree-Based Models (e.g., XGBoost, Random Forest): Both one-hot encoding and label encoding work well. However, target encoding may provide an advantage if we suspect a strong relationship between categories and the target.
        For Linear Models (e.g., Logistic Regression): one-hot encoding to prevent the model from assuming ordinal relationships.
        For High-Dimensional Categorical Variables: target encoding or frequency encoding to reduce the number of columns.

## 1. Encoding

In [3]:
# Visualize unique values per column to check the categories
columns_to_check = [
    'Result_offer',
    'Bid_Notbid',
    'Client_type',
    'Market_segment',
    'Client_BU',
    'Country',
    'Client_importance',
    'Coverage_model',
    'Result_offer_binary'
]
					
for column in columns_to_check:
    unique_values = df_normalized[column].unique()
    unique_count = df_normalized[column].nunique()
    print(f"* Column '{column}':")
    print(f"  - Unique values ({unique_count}): {unique_values}\n")

* Column 'Result_offer':
  - Unique values (2): ['Lost' 'Won']

* Column 'Bid_Notbid':
  - Unique values (1): ['Bid']

* Column 'Client_type':
  - Unique values (13): ['Contractor' 'B2B Distribution' 'End User' 'Panel Builder' 'Design Firm'
 'Digital & Service Provider' 'Not informed'
 'Original Equipment Manufacturer' 'Electrician' 'System Integrator'
 'IT Channel Partner' 'Internal Schneider Entity' 'DIY & RETAILER']

* Column 'Market_segment':
  - Unique values (125): ['BDZ' 'E34' 'I9Z' 'I44' 'I20' 'IDZ' 'Not informed' 'E42' 'I91' 'E08'
 'E07' 'B41' 'I4F' 'E9F' 'I24' 'B47' 'B12' 'B15' 'EN2' 'E09' 'I46' 'E37'
 'BD1' 'I9Y' 'D91' 'EN1' 'B44' 'B17' 'E04' 'E9B' 'E46' 'I37' 'D11' 'EN3'
 'E9D' 'E05' 'B9A' 'I4B' 'E38' 'R02' 'R01' 'I34' 'B14' 'IT3' 'E33' 'I50'
 'I40' 'ID2' 'I25' 'B38' 'ID3' 'B13' 'ID1' 'B93' 'PW1' 'BG2' 'I43' 'E14'
 'ID4' 'I28' 'B45' 'E9C' 'I52' 'I21' 'B91' 'I51' 'B90' 'I60' 'B9B' 'BD3'
 'BR4' 'D95' 'BD4' 'I48' 'I93' 'BDR' 'ID5' 'I47' 'E21' 'BG1' 'I32' 'E01'
 'I42' 'E4A' 'I4

In [4]:
#ENCODING

# Copy the dataset
df_encoded = df_normalized.copy()

# Encoding and model preparation

# A. Label Encoding for low cardinality columns
label_columns = ['Country', 'Coverage_model']
label_encoders = {}
for col in label_columns:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    label_encoders[col] = le

# B. Ordinal Encoding for 'Client_importance'
client_importance_mapping = {
    'MA3': 3, 'MA2': 2, 'MA1': 1,
    'MG3': 3, 'MG2': 2, 'MG1': 1,
    'MS3': 3, 'MS2': 2, 'MS1': 1,
    'Not informed': 0
}
df_encoded['Client_importance'] = df_encoded['Client_importance'].map(client_importance_mapping)

# C. Target Encoding for medium cardinality columns
target_encoder = ce.TargetEncoder(cols=['Client_type'])
df_encoded = target_encoder.fit_transform(df_encoded, df_encoded['Result_offer_binary'])


# D. Frequency Encoding for high cardinality columns
# 1. 'Market_segment'
for col in ['Market_segment']:
    freq_encoding = df_encoded[col].value_counts().to_dict()
    df_encoded[col] = df_encoded[col].map(freq_encoding)

# 2. 'Client_BU'
client_bu_freq = df_encoded['Client_BU'].value_counts().to_dict()
df_encoded['Client_BU'] = df_encoded['Client_BU'].map(client_bu_freq)


# E1. Encoding SR_codigo

# Create a dictionary to encode 'SR_codigo'
unique_values = df_encoded['SR_codigo'].unique()
sr_codigo_encoder = {value: idx for idx, value in enumerate(unique_values)}
 
# Replace the values in the 'SR_codigo' column with their encoded values
df_encoded['SR_codigo'] = df_encoded['SR_codigo'].map(sr_codigo_encoder)

# E. Drop 'Result_offer' as it's redundant with 'Result_offer_binary', and Bid_Notbid as it is not a predictor for us
df_encoded = df_encoded.drop(columns=['Result_offer', 'Bid_Notbid', 'SR_codigo'])

# Display the encoded dataframe
# print(df_encoded.head())
df_encoded.head()

,Amount_DE,Amount_HD,Amount_ID,Amount_OTHER,Amount_PP,Amount_PS,Amount_SP,Total_Amount,Resolution_time,Client_type,Market_segment,Client_BU,Country,Client_importance,Coverage_model,Result_offer_binary
0,0.001945,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000718,0.078611,0.456166,1037,3473,1,3,6,0
1,0.002045,0.0,0.1,0.0,0.02803,0.020478,0.0,0.021526,0.235466,0.456166,137,2861,1,3,6,1
2,0.003382,0.0,0.0,0.0,0.00000,0.000005,0.0,0.001252,0.032176,0.456166,248,3473,1,3,6,0
3,0.003382,0.0,0.0,0.0,0.00000,0.000005,0.0,0.001252,0.025229,0.456166,248,3473,1,3,6,0
4,0.003382,0.0,0.0,0.0,0.00000,0.000005,0.0,0.001252,0.025229,0.456166,248,3473,1,3,6,0


In [ ]:
# SEE WHAT WAS ENCONDED - Retrieve encodings

# A. Label Encoding mappings
print("Label Encoding Mappings:")
for col, le in label_encoders.items():
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print(f"Column '{col}': {mapping}")

# B. Ordinal Encoding mappings
print("\nOrdinal Encoding Mapping for 'Client_importance':")
print(client_importance_mapping)

# C. Target Encoding (Client_type)
print("\nTarget Encoding: Client_type")
# Target encoding doesn't create an explicit mapping since it's based on mean target values.
# To inspect the encoded values, you can group by the original column to see the target means:
original_client_type_mapping = df_normalized.groupby('Client_type')['Result_offer_binary'].mean().to_dict()
print(original_client_type_mapping)

# D. Frequency Encoding mappings
print("\nFrequency Encoding Mappings:")
# Market_segment
market_segment_freq = df_normalized['Market_segment'].value_counts().to_dict()
print("Market_segment:", market_segment_freq)

# Client_BU
client_bu_freq = df_normalized['Client_BU'].value_counts().to_dict()
print("Client_BU:", client_bu_freq)

# E1. SR_codigo
# Print the encoded dictionary for reference
print("Encoded values stored in the dictionary:")
for key, value in sr_codigo_encoder.items():
    print(f"{key}: {value}")


# E. Confirm dropped columns
print("\nDropped Columns: ['Result_offer', 'SR_codigo', 'Bid_Notbid']")


Label Encoding Mappings:
Column 'Country': {'AD': 0, 'ES': 1, 'GB': 2, 'Not informed': 3, 'PT': 4}
Column 'Coverage_model': {'Delegated to Partner': 0, 'Inside Sales': 1, 'Marketing': 2, 'Not Covered': 3, 'Not informed': 4, 'Outsourced Sales': 5, 'Sales Rep': 6}

Ordinal Encoding Mapping for 'Client_importance':
{'MA3': 3, 'MA2': 2, 'MA1': 1, 'MG3': 3, 'MG2': 2, 'MG1': 1, 'MS3': 3, 'MS2': 2, 'MS1': 1, 'Not informed': 0}

Target Encoding: Client_type
{'B2B Distribution': 0.5342105263157895, 'Contractor': 0.45616605616605616, 'DIY & RETAILER': 0.6111111111111112, 'Design Firm': 0.5272727272727272, 'Digital & Service Provider': 0.6106194690265486, 'Electrician': 0.47555555555555556, 'End User': 0.7135789832821563, 'IT Channel Partner': 0.5948275862068966, 'Internal Schneider Entity': 0.4, 'Not informed': 0.5458333333333333, 'Original Equipment Manufacturer': 0.5647668393782384, 'Panel Builder': 0.5777202072538861, 'System Integrator': 0.58203125}

Frequency Encoding Mappings:
Market_segme

**Summary of encoding**
1. Label Encoding for Low Cardinality Columns
    Columns: Bid_Notbid, Country, Coverage_model
    Method: LabelEncoder
    Reasoning: 
    These columns have a small number of categories (3–7), so Label Encoding is efficient and interpretable.
    Since these columns are unlikely to have a strong ordinal relationship, Label Encoding works well for low-cardinality features.
    The use of a dictionary to store the encoders (label_encoders) is good practice, enabling us to reverse-transform if needed.
2. Ordinal Encoding for Client_importance
    Mapping: Clear prioritization of importance levels (3 > 2 > 1 > 0 for Not informed).
    Reasoning:
    This method captures the natural order in Client_importance.
    Assigning Not informed a value of 0 is reasonable since its lack of information makes it less predictive.
3. Target Encoding for Medium Cardinality Columns
    Column: Client_type (13 categories)
    Reasoning:
    Target Encoding is a good choice for columns where the target variable (Result_offer_binary) is likely influenced by the feature.
    Using the mean of Result_offer_binary for each Client_type ensures the model captures this relationship.
    Ensuring this encoding is done using cross-validation is critical to prevent data leakage.
4. Frequency Encoding for High Cardinality Columns
    Column: Market_segment (126 categories)
    Reasoning:
    Frequency Encoding reduces the cardinality without introducing target leakage.
    Mapping counts of each category makes the data compact while preserving the relative importance of categories based on their prevalence.
5. Dropping Redundant/Unique Columns
    Dropped Columns: SR_codigo, Result_offer, Bid_Notbid
    Reasoning:
    SR_codigo is unique identifier and does not offer predictive power.
    Result_offer is redundant with the binary column Result_offer_binary, which is a more ML-friendly representation.
    Bid_Notbid is not a real variable for prediction.


**TEMPORARY EXCEPTION**
(SEE E1) We temporarily encoded SR_codigo as a workaround to match the predictions for Pipeline or Not_decided_yet. We will investigate the correct method to match the results with the original offers without encoding SR_codigo.

## 2. Correlation Check:

After encoding, check the correlation of predictors with Result_offer_binary to validate the effectiveness of the encodings.

In [6]:
# Correlation check

correlation_matrix = df_encoded.corr()
target_corr = correlation_matrix['Result_offer_binary'].sort_values(ascending=False)
print(target_corr)

Result_offer_binary    1.000000
Client_type            0.233203
Country                0.147328
Coverage_model         0.056928
Amount_SP              0.032366
Amount_PP             -0.006643
Amount_OTHER          -0.015636
Client_importance     -0.018011
Amount_HD             -0.020809
Amount_DE             -0.031722
Total_Amount          -0.039292
Client_BU             -0.042743
Amount_ID             -0.044491
Amount_PS             -0.051130
Resolution_time       -0.099524
Market_segment        -0.126515
Name: Result_offer_binary, dtype: float64


**Analysis**
1. Target Variable (Result_offer_binary)
    The correlation values measure the strength and direction of the linear relationship between Result_offer_binary and other features.
    The correlations are relatively weak, with no variable showing a strong correlation with Result_offer_binary. This suggests that the relationship between the features and the target variable might not be linear or there may be other complex relationships.

2. Top Positively Correlated Variables
    Client_type (0.236):
    This shows the strongest positive correlation with Result_offer_binary. Certain client types may be more likely to result in "Won" offers.
    Country (0.147):
    A modest positive correlation indicates that location might play a role in determining the outcome.
    Coverage_model (0.059):
    Slight positive correlation suggests that the sales coverage model might impact outcomes, though the effect is weak.

3. Negatively Correlated Variables
    Bid_Notbid (-0.188):
    This is the most negatively correlated variable, suggesting that "Bid" or "Not_bid" decisions are moderately predictive of offer outcomes. For example, "Not_bid" might correlate more strongly with "Lost". As this is not a real variable to be considered, we excluded it from the df for ML training.
    Market_segment (-0.123):
    Indicates that some market segments may correlate with a lower likelihood of "Won" offers.
    Resolution_time (-0.056):
    Longer resolution times might slightly reduce the chances of winning offers.
    Amount_PS (-0.050):
    Indicates that higher values in this specific amount category may slightly reduce the likelihood of a "Won" outcome.

4. Low Correlation Variables
    Variables like Total_Amount, Client_BU, and Amount columns generally have very weak correlations with the target variable. This could indicate:
        Non-linear relationships.
        Insignificant influence on the target variable.
        Potential noise in the data.


**Conclusion**
Model Selection: Since no variable shows strong linear correlation, tree-based models like Random Forest or XGBoost would work better, as they can capture non-linear relationships and interactions between variables.


## 3. Feature Importance Analysis

Use Random Forest or XGBoost model to analyze feature importance, ensuring the encodings lead to meaningful predictors.

In [7]:
from xgboost import XGBClassifier

model = XGBClassifier()
X = df_encoded.drop(columns=['Result_offer_binary'])
y = df_encoded['Result_offer_binary']
model.fit(X, y)
importance = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
importance = importance.sort_values(by='Importance', ascending=False)
print(importance)

              Feature  Importance
9         Client_type    0.149041
7        Total_Amount    0.119120
12            Country    0.082128
0           Amount_DE    0.068512
5           Amount_PS    0.068322
4           Amount_PP    0.061574
14     Coverage_model    0.059607
6           Amount_SP    0.058246
2           Amount_ID    0.057921
10     Market_segment    0.051187
1           Amount_HD    0.050900
11          Client_BU    0.050655
13  Client_importance    0.048047
8     Resolution_time    0.043949
3        Amount_OTHER    0.030790


**Analysis**

Client Characteristics Matter Most:
    Client_type, Country, and Total_Amount dominate, highlighting the importance of understanding the client and transaction context.
Transaction Amount Features:
    The variety of Amount_* features collectively add predictive power, indicating that individual product or service lines influence success differently.
Coverage and Market Insights:
    Coverage_model and Market_segment are moderately important, suggesting differences in approach or market conditions.


## 4. Cross-Validation

Ensure the model performance is robust by using cross-validation:

In [8]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=5, scoring='roc_auc')
print(f"Average AUC: {scores.mean():.4f}")

Average AUC: 0.7879


**Analysis**
Interpretation of the AUC Score
AUC measures the ability of the model to distinguish between the positive and negative classes.
AUC ranges from 0 to 1:
0.5: The model is no better than random guessing.
>0.7: Indicates a good model.
>0.8: Indicates a strong model.
>0.9: Indicates an excellent model.

In [10]:
#Export to CSV
df_encoded.to_csv('04_df_encoded_wonlost.csv', index=False)